In [14]:
from pathlib import Path, PurePath

Path("E:\간판여행2\images.txt").stem
PurePath("E:/간판여행2/images.txt").stem

'images'

In [19]:
Path("hello.txt").rename("ss")

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: 'hello.txt' -> 'ss'

In [11]:
glob.glob("E:\간판여행2\images\*\*.jpg")

[]

In [20]:
import os 


for i, img in enumerate(Path("E:\간판여행2\images").glob("*")):
    img.rename(f"E:\간판여행2\images\{i+1}.jpg")

OSError: [WinError 17] 시스템은 파일을 다른 디스크 드라이브로 옮길 수 없습니다: 'E:\\간판여행2\\images\\-_-_000001_jpg.rf.2ee3cabb8a6e90693c76ce925a845a59.jpg' -> '1.jpg'

In [4]:
from paddle.inference import create_predictor


In [15]:
import paddle
gpu_available  = paddle.device.is_compiled_with_cuda()
print("GPU available:", gpu_available)

grep: warning: GREP_OPTIONS is deprecated; please use an alias or script


GPU available: True


In [69]:
model = paddle.nn.Linear(3, 1)
print(model.state_dict())
paddle.save(model.state_dict(), "/home/test/model.pdiparamsss")

OrderedDict([('weight', Parameter containing:
Tensor(shape=[3, 1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[-0.82215434],
        [ 0.03054924],
        [ 0.95529938]])), ('bias', Parameter containing:
Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [0.]))])


In [72]:
paddle.save(model, "/home/test/model.pdmodel")

ValueError: paddle do not support saving `paddle.nn.Layer` object.

In [71]:
model = paddle.nn.Linear(3, 1)
print(model.state_dict())
model.load_dict(paddle.load("/home/test/model.pdiparamsss"))
print(model.state_dict())

print(type(model.state_dict()))



OrderedDict([('weight', Parameter containing:
Tensor(shape=[3, 1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[ 0.06659265],
        [ 0.62681735],
        [-0.29351181]])), ('bias', Parameter containing:
Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [0.]))])
OrderedDict([('weight', Parameter containing:
Tensor(shape=[3, 1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[-0.82215434],
        [ 0.03054924],
        [ 0.95529938]])), ('bias', Parameter containing:
Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [0.]))])
<class 'collections.OrderedDict'>


In [ ]:
import collections.OrderedDict

In [ ]:

model.load_dict()

In [60]:
model = paddle.nn.Linear(3, 1)
# model.save("")
model.state_dict()

paddle.save(model.state_dict(), "/home/test/model.pdparams")
paddle.save(model.state_dict(), "/home/test/model.pdparams")
# import paddle.fluid.io

# paddle.fluid.io.save_inference_model(model, f)

In [80]:
import paddle

# paddle.nn.Linear(3, 1).save_inference_model("/home/test/test.pdmodel")


print(type(paddle.nn.Linear(3, 1)))

model = paddle.nn.layer.common.Linear(3, 1)

paddle.jit.save(model, "/home/test/test.pdmodel")



<class 'paddle.nn.layer.common.Linear'>


ValueError: No valid transformed program for function: forward(input), input_spec: None.
	    Please specific `input_spec` in `@paddle.jit.to_static` or feed input tensor to call the decorated function at once.


In [74]:
import paddle

config = paddle.inference.Config("/home/test/model.mobilenet_v1.pdmodel", "./mobilenet_v1.pdiparams")

# 根据 config 创建 predictor
predictor = paddle.inference.create_predictor(config)

RuntimeError: (NotFound) Cannot open file ./mobilenet_v1.pdmodel, please confirm whether the file is normal.
  [Hint: Expected static_cast<bool>(fin.is_open()) == true, but received static_cast<bool>(fin.is_open()):0 != true:1.] (at ../paddle/fluid/inference/api/analysis_predictor.cc:2364)


In [36]:
import paddle

class MyLayer(paddle.nn.Layer):
    def __init__(self):
        super().__init__()
        self._linear1 = 
        print(self._linear1.weight)
        pass
    
    def forward(self, input):
        return self._linear1(input)
    
mylayer = MyLayer()
x = paddle.randn([3], "float32")
print(x)
mylayer.eval()  # set mylayer._dropout to eval mode
out = mylayer(x)
print(out)
mylayer.train()  # set mylayer._dropout to train mode
out = mylayer(x)
print(out)

Parameter containing:
Tensor(shape=[3, 1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[ 0.62206370],
        [-0.53300339],
        [ 0.59778345]])
Tensor(shape=[3], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [-1.39317989,  0.76278561, -0.08508420])
Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [-1.32407594])
Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [-1.32407594])


Tensor(shape=[3], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [-1.39317989,  0.76278561, -0.08508420])
Tensor(shape=[3], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [ 0.62206370, -0.53300339,  0.59778345])


Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [-1.32407594])

In [31]:
x = paddle.randn([[1,1,1], [2,2,2]], "float32")

ValueError: (InvalidArgument) gaussian(): argument (position 1) must be list of int, but got list at pos 0 (at ../paddle/fluid/pybind/op_function_common.cc:475)


In [ ]:
layer = paddle.nn.Linear(3, 1)
x = 

In [20]:
import paddle

class MyLayer(paddle.nn.Layer):
    def __init__(self):
        super().__init__()
        self._linear1 = paddle.nn.Linear(1, 1)

    def forward(self, input):
        temp = self._linear1(input)
        return temp

x = paddle.randn([10, 1], 'float32')
mylayer = MyLayer()

In [15]:
import paddle

class MyLayer(paddle.nn.Layer):
    def __init__(self):
        super().__init__()
        self._linear1 = paddle.nn.Linear(1, 1)

    def forward(self, input):
        temp = self._linear1(input)
        return temp

x = paddle.randn([10, 1], 'float32')
mylayer = MyLayer()
mylayer.eval()  # set mylayer._dropout to eval mode
out = mylayer(x)
mylayer.train()  # set mylayer._dropout to train mode
out = mylayer(x)

In [84]:
import paddle.nn

paddle.nn.Linear((1, 2), [3])

TypeError: '>' not supported between instances of 'tuple' and 'int'

In [108]:
IMAGE_SIZE = 1
CLASS_NUM = 3

class LinearNet(paddle.nn.Layer):
    def __init__(self):
            super(LinearNet, self).__init__()
            self._linear= paddle.nn.Linear(IMAGE_SIZE, CLASS_NUM)

    @paddle.jit.to_static
    def forward(self, x):
        return self._linear(x)

layer= LinearNet()

path= "/home/test/model/inference"
paddle.jit.save(layer, path)

TypeError: The type of 'function' should be a function or method, but received type.

In [96]:
import paddle

@paddle.jit.to_static
def func1(x):
    return x+1

x = paddle.ones([2, 2])

func1(x)

paddle.jit.save(func1, "/home/test/model.pdmodel")

/usr/local/lib/python3.7/dist-packages/paddle/jit/api.py:945: UserWarning: What you save is a function, and `jit.save` will generate the name of the model file according to `path` you specify. When loading these files with `jit.load`, you get a `TranslatedLayer` whose inference result is the same as the inference result of the function you saved.
  'What you save is a function, and `jit.save` will generate the name of the model file according to `path` you specify. When loading these files with `jit.load`, you get a `TranslatedLayer` whose inference result is the same as the inference result of the function you saved.'
/usr/local/lib/python3.7/dist-packages/paddle/static/io.py:994: UserWarning: no variable in your model, please ensure there are any variables in your model to save
  "no variable in your model, please ensure there are any variables in your model to save"


In [98]:
model = paddle.jit.load("/home/test/model.pdmodel")
model.state_dict()

OrderedDict()

In [95]:
import paddle

def func2(x):
    return x+1

x = paddle.ones([2, 2])

func2(x)

Tensor(shape=[2, 2], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [[2., 2.],
        [2., 2.]])

In [113]:
import paddle

class MyLayer(paddle.nn.Layer):
    def __init__(self):
        super().__init__()
        self.layer1 = paddle.nn.Linear(1, 3)
        self.layer2 = paddle.nn.Linear(3, 1)
    
    @paddle.jit.to_static
    def forward(self, input):
        input = self.layer1(input)
        input = self.layer2(input)
        return input
    

model = MyLayer()
x = paddle.randn([3, 1])

y = model(x)
print(y)


paddle.jit.save(model, "/home/test/model/model1.pamodel")
model2 = paddle.jit.load("/home/test/model/model1.pamodel")
y2 = model2(x)
print(y2)

Tensor(shape=[3, 1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[-0.20148109],
        [ 0.53908861],
        [-0.21063973]])
Tensor(shape=[3, 1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[-0.20148109],
        [ 0.53908861],
        [-0.21063973]])


In [114]:
paddle.nn.Liear

import paddle.nn



AttributeError: module 'paddle.nn' has no attribute 'Liear'

In [121]:
model = paddle.nn.Linear(in_features=None, out_features=None)


TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [132]:
model = paddle.nn.Linear(1, 2)


x = paddle.randn([30, 30])
# model(x)
model.forward(x)

ValueError: (InvalidArgument) Input(Y) has error dim.Y'dims[0] must be equal to 30But received Y'dims[0] is 1
  [Hint: Expected y_dims[y_ndim - 2] == K, but received y_dims[y_ndim - 2]:1 != K:30.] (at ../paddle/phi/kernels/impl/matmul_kernel_impl.h:324)
  [operator < linear > error]

In [131]:
paddle.in_dynamic_mode()

True

Parameter containing:
Tensor(shape=[1, 1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[0.01287939]])
Parameter containing:
Tensor(shape=[1, 1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[0.01287939]])


In [149]:
model.weight.gradient

<bound method monkey_patch_tensor.<locals>.gradient of Parameter containing:
Tensor(shape=[1, 1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[0.01287939]])>

In [167]:
model = paddle.nn.Linear(1, 1)
x = paddle.to_tensor([2], dtype = "float32")
model.weight.gradient

model.weight.gradient
model.weight.clear_gradient()
model.weight.gradient

<bound method monkey_patch_tensor.<locals>.gradient of Parameter containing:
Tensor(shape=[1, 1], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[1.04335737]])>

In [8]:
import paddle
paddle.fluid.io.save_inference_model()

TypeError: missing a required argument: 'dirname'